# 14. 프로젝트 - Movielens 영화 추천 실습  
MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작
활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터

# 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [6]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
total = pd.merge(ratings,movies,how='left', on=['movie_id'])
total.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


# 2) 분석해 보기

In [9]:
# 영화 개수
ratings['movie_id'].nunique()

3628

In [10]:
# 사용자 수
ratings['user_id'].nunique()

6039

In [11]:
# 가장 인기 있는 영화 30개(인기순)
movie_count = total.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

# 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 주기

In [12]:
data_df = total.copy()

In [13]:
# 사용하는 컬럼만 남겨줍니다.
data_df = data_df[['user_id','title', 'counts']]
data_df.tail()

,user_id,title,counts
836473,6040,Platoon (1986),3
836474,6040,"Crying Game, The (1992)",5
836475,6040,Welcome to the Dollhouse (1995),5
836476,6040,Sophie's Choice (1982),4
836477,6040,E.T. the Extra-Terrestrial (1982),4


In [14]:
movies['title'] = movies['title'].str.lower() # 검색을 쉽게 하기 위해 영화제목을 소문자로 바꿔줍시다.
data_df.head(10)

,user_id,title,counts
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,1,James and the Giant Peach (1996),3
2,1,My Fair Lady (1964),3
3,1,Erin Brockovich (2000),4
4,1,"Bug's Life, A (1998)",5
5,1,"Princess Bride, The (1987)",3
6,1,Ben-Hur (1959),5
7,1,"Christmas Story, A (1983)",5
8,1,Snow White and the Seven Dwarfs (1937),4
9,1,"Wizard of Oz, The (1939)",4


In [15]:
# 본인이 좋아하시는 영화 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['American Beauty (1999))' , 'Matrix, The (1999)' ,'Forrest Gump (1994)' ,'Back to the Future (1985)' ,'Saving Private Ryan (1998)']

# 'kmin'이라는 user_id가 위 영화를 5회씩 시청했다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['kmin']*5, 'title':  my_favorite, 'counts':[5]*5})

if not data_df.isin({'user_id':['kmin']})['user_id'].any():  # user_id에 'kmin'이라는 데이터가 없다면
    data_df = data_df.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data_df.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,counts
836473,6040,Platoon (1986),3
836474,6040,"Crying Game, The (1992)",5
836475,6040,Welcome to the Dollhouse (1995),5
836476,6040,Sophie's Choice (1982),4
836477,6040,E.T. the Extra-Terrestrial (1982),4
0,kmin,American Beauty (1999)),5
1,kmin,"Matrix, The (1999)",5
2,kmin,Forrest Gump (1994),5
3,kmin,Back to the Future (1985),5
4,kmin,Saving Private Ryan (1998),5


# 4) CSR matrix를 직접 만들어 보기

In [16]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data_df['user_id'].unique()
title_unique = data_df['title'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [17]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['kmin'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(title_to_idx['Platoon (1986)'])

6039
1030


In [18]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data_df['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data_df):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data_df['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data_df['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data_df):
    print('title column indexing OK!!')
    data_df['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data_df

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,3628,5
1,6039,124,5
2,6039,160,5
3,6039,22,5


In [19]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data_df['user_id'].nunique()
num_movie = data_df['title'].nunique()

csr_data = csr_matrix((data_df['counts'], (data_df.user_id, data_df.title)), shape= (num_user, num_movie))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델 및 훈련

In [22]:
#Matrix Factorization 모델을 implicit 패키지를 사용하여 학습
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [23]:
#Matrix Factorization 모델을 implicit 패키지를 사용하여 학습
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [27]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [24]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [28]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보기.

In [29]:
kmin, matrix = user_to_idx['kmin'], title_to_idx['Matrix, The (1999)']
kmin_vector, matrix = als_model.user_factors[kmin], als_model.item_factors[matrix]

In [30]:
# kmin과 matrix를 내적하는 코드
np.dot(kmin_vector,matrix)

0.46521512

In [31]:
forrest = title_to_idx['Forrest Gump (1994)']
forrest_vector = als_model.item_factors[forrest]
np.dot(kmin_vector, forrest_vector)

0.60313314

In [32]:
another= title_to_idx["Platoon (1986)"]
another_vector = als_model.item_factors[another]
np.dot(kmin_vector, another_vector) 

-0.016186211

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 보기

## AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 영화를 찾기  

In [33]:
idx_to_title = {v:k for k,v in title_to_idx.items()}
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [34]:
get_similar_movie('Truman Show, The (1998)')

['Truman Show, The (1998)',
 'Apollo 13 (1995)',
 'Quiz Show (1994)',
 'People vs. Larry Flynt, The (1996)',
 "Mr. Holland's Opus (1995)",
 'Philadelphia (1993)',
 'Good Will Hunting (1997)',
 'Fried Green Tomatoes (1991)',
 'Primary Colors (1998)',
 'Rainmaker, The (1997)']

In [35]:
get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Babe (1995)',
 'Groundhog Day (1993)',
 'Pleasantville (1998)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)"]

In [36]:
get_similar_movie('Terminator, The (1984)')

['Terminator, The (1984)',
 'Aliens (1986)',
 'Die Hard (1988)',
 'Alien (1979)',
 'Predator (1987)',
 'Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Blade Runner (1982)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Total Recall (1990)']

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 보기.

## AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받기  
## filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument  

In [37]:
user = user_to_idx['kmin']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(87, 0.50942576),
 (92, 0.42237204),
 (23, 0.40884602),
 (64, 0.36607528),
 (107, 0.3316991),
 (121, 0.31256828),
 (117, 0.31240308),
 (44, 0.30866995),
 (110, 0.2843248),
 (141, 0.28307623),
 (157, 0.27977872),
 (175, 0.260502),
 (40, 0.25725758),
 (200, 0.23885411),
 (60, 0.22883825),
 (154, 0.22717768),
 (248, 0.22184671),
 (62, 0.21659407),
 (38, 0.21615188),
 (243, 0.21314801)]

In [38]:
[idx_to_title[i[0]] for i in movie_recommended]

['Braveheart (1995)',
 'Terminator 2: Judgment Day (1991)',
 "Schindler's List (1993)",
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Jurassic Park (1993)',
 'Silence of the Lambs, The (1991)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Groundhog Day (1993)',
 'Fugitive, The (1993)',
 'Shawshank Redemption, The (1994)',
 'Men in Black (1997)',
 'Toy Story (1995)',
 'Terminator, The (1984)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'As Good As It Gets (1997)',
 'Good Will Hunting (1997)',
 'Total Recall (1990)',
 'Sixth Sense, The (1999)',
 'Ghostbusters (1984)']

# 9) 회고  
## 넷플릭스에 들어가면 나에게 맞는 추천 알고리즘이 뜬다. 많은 기술들로 사용자에 맡는 추천을 한다 .그점이 좋앗고 해커톤에도 넷플릭스를 선택하여 앞으로가 기대된다.  
